In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 5,
    "exp_name": ct.generate_readable_short_id("Paper_v1_Real30"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 10,
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt", "RF_large",
                      "Platt", "ISO", "Beta", "CRF", "VA",
                      "tlr", "Rank", "Elkan",                      
                      ],
    
    "metrics": ["acc", "brier", "ece", "logloss"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false

    "exp_key": "data_name",
    "exp_values": [
                  "cm1",
                  "datatrieve",
                  "kc1_class_level_defectiveornot",
                  "kc1",
                  "kc2",
                  "kc3",
                  "pc1",
                  "spect",
                  "spectf",
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                  "bank", 
                #   "ionosphere", 
                #   "HRCompetencyScores",
                #   "spambase", 
                #   "QSAR", 
                #   "diabetes", 
                #   "breast", 
                #   "SPF",
                #   "hillvalley",
                #   "pc4",
                #   "scene",
                #   "Sonar_Mine_Rock_Data",
                #   "Customer_Churn",
                #   "jm1",
                #   "eeg",
                #   "phoneme",
                  # "madelon",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param cm1 done
exp_param datatrieve done
exp_param kc1_class_level_defectiveornot done
exp_param kc1 done
exp_param kc2 done
exp_param kc3 done
exp_param pc1 done
exp_param spect done
exp_param spectf done
exp_param vertebral done
exp_param wilt done
exp_param parkinsons done
exp_param heart done
exp_param wdbc done
exp_param bank done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
tables["brier"].round(4)


,RF_d,RF_opt,Platt,ISO,Beta,CRF,VA,tlr,Rank,Elkan,RF_large
Data,,,,,,,,,,,
cm1,0.0878,0.0848,0.0883,0.0881,0.0881,0.0887,0.0894,0.0980,0.0898,0.1889,0.0833
datatrieve,0.0792,0.0870,0.1004,0.1428,0.1398,0.0843,0.1221,0.0832,0.0873,0.1840,0.0834
kc1_class_level_defectiveornot,0.1809,0.1833,0.1726,0.2009,0.1856,0.1747,0.1847,0.1837,0.1902,0.2578,0.1716
kc1,0.1120,0.1073,0.1105,0.1126,0.1097,0.1090,0.1103,0.1377,0.1138,0.2211,0.1063
kc2,0.1277,0.1197,0.1214,0.1259,0.1208,0.1202,0.1229,0.1422,0.1256,0.2256,0.1133
kc3,0.0776,0.0792,0.0797,0.0780,0.0775,0.0782,0.0807,0.0880,0.0767,0.1597,0.0734
pc1,0.0543,0.0517,0.0563,0.0565,0.0570,0.0521,0.0539,0.0655,0.0558,0.1194,0.0513
spect,0.1377,0.1280,0.1361,0.1576,0.1440,0.1253,0.1480,0.1418,0.1913,0.1697,0.1206
spectf,0.1406,0.1269,0.1340,0.1432,0.1339,0.1322,0.1316,0.1449,0.1471,0.1698,0.1218


In [6]:
ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}")

metric acc
Friedman Test Statistic: 63.00458015267173
P-value: 9.744007697606032e-10
The differences between groups are significant.
metric brier
Friedman Test Statistic: 96.84848484848487
P-value: 2.324140356138094e-16
The differences between groups are significant.
metric ece
Friedman Test Statistic: 63.27272727272725
P-value: 8.662435601016003e-10
The differences between groups are significant.
metric logloss
Friedman Test Statistic: 114.56969696969702
P-value: 6.37238608081919e-20
The differences between groups are significant.
